In [1]:
from pathlib import Path
import pandas as pd
csv_files = list((Path().glob('*.csv')))
dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    split = str(file).split('_')
    architecture = split[0]
    tokenizer = split[-2]
    random_weights = False
    if('RANDOM' in split):
        random_weights = True
        
    df.insert(0, 'random_weights',random_weights)
    df.insert(0, 'tokenizer',tokenizer)
    df.insert(0, 'architecture',architecture)
    
    df['file_name'] = file
    dfs.append(df)
    # print(df.head())

main_df = pd.concat(dfs, ignore_index=True)

In [2]:
dsets = list(main_df.dataset.unique())

In [3]:
# main_df.head()

In [4]:
dfs = []
for dataset_index in range(7):
    df = main_df[
        # (main_df['random_weights']==True) & 
        # (main_df['architecture']=='DEBERTA') & 
        (main_df['dataset']==dsets[dataset_index])
    ][['architecture','dataset','tokenizer','random_weights','test_acc', 'test_f1', 'test_auroc_weighted','test_pr_auc']]\
    .sort_values('tokenizer')\
    .style\
    .hide(axis='index')\
    .format({"dataset": lambda x:x.lower()})\
    .background_gradient(subset=['test_acc', 'test_f1', 'test_auroc_weighted','test_pr_auc'], cmap='Blues')
    # .highlight_max(subset=['test_auroc_weighted'],color='lightgreen')\
    # .highlight_min(subset=['test_auroc_weighted'],color='#cd4f39')\
    dfs.append(df)
    # main_df.iloc[main_df.groupby(['dataset','tokenizer'])['test_auroc_weighted'].idxmax()]
    # main_df.iloc[main_df.groupby('dataset')['test_acc'].idxmax()]
    


In [5]:
dfs[0]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
BERT,demo_coding_vs_intergenomic_seqs,DNABERTtokenizer,False,0.924600,0.925936,0.978707,0.978504
DEBERTA,demo_coding_vs_intergenomic_seqs,DNABERTtokenizer,True,0.901200,0.901492,0.963793,0.961481
BERT,demo_coding_vs_intergenomic_seqs,DNABERTtokenizer,True,0.900920,0.900606,0.963096,0.960968
BERT,demo_coding_vs_intergenomic_seqs,DNABERTtokenizer,True,0.899560,0.901425,0.963367,0.961143
DEBERTA,demo_coding_vs_intergenomic_seqs,DNABERTtokenizer,False,0.918320,0.918587,0.973802,0.973179
BERT,demo_coding_vs_intergenomic_seqs,DNABERTtokenizer,False,0.924600,0.925936,0.978707,0.978504
BERT,demo_coding_vs_intergenomic_seqs,DNABERTtokenizer,True,0.900920,0.900606,0.963096,0.960968
BERT,demo_coding_vs_intergenomic_seqs,Kmer7tokenizer,True,0.901520,0.902826,0.963782,0.962473
BERT,demo_coding_vs_intergenomic_seqs,Kmer7tokenizer,True,0.898840,0.901584,0.963771,0.962069
DEBERTA,demo_coding_vs_intergenomic_seqs,Kmer7tokenizer,True,0.901000,0.899452,0.963838,0.962015


In [6]:
dfs[1]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
BERT,demo_human_or_worm,DNABERTtokenizer,False,0.965800,0.965733,0.995246,0.995429
BERT,demo_human_or_worm,DNABERTtokenizer,True,0.953480,0.952710,0.991099,0.991605
DEBERTA,demo_human_or_worm,DNABERTtokenizer,False,0.959320,0.958250,0.995005,0.995096
BERT,demo_human_or_worm,DNABERTtokenizer,False,0.965800,0.965733,0.995246,0.995429
BERT,demo_human_or_worm,DNABERTtokenizer,True,0.953480,0.952710,0.991099,0.991605
DEBERTA,demo_human_or_worm,DNABERTtokenizer,True,0.954600,0.954912,0.991370,0.991745
BERT,demo_human_or_worm,DNABERTtokenizer,True,0.954080,0.953925,0.990508,0.991099
DEBERTA,demo_human_or_worm,Kmer7tokenizer,False,0.958560,0.957510,0.994299,0.994550
BERT,demo_human_or_worm,Kmer7tokenizer,True,0.955720,0.955597,0.991495,0.991811
DEBERTA,demo_human_or_worm,Kmer7tokenizer,True,0.955400,0.955622,0.991569,0.991967


In [7]:
dfs[2]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
BERT,human_enhancers_cohn,DNABERTtokenizer,False,0.741940,0.749476,0.828015,0.818516
BERT,human_enhancers_cohn,DNABERTtokenizer,True,0.726540,0.733819,0.800259,0.789622
DEBERTA,human_enhancers_cohn,DNABERTtokenizer,False,0.731434,0.723801,0.813430,0.803940
BERT,human_enhancers_cohn,DNABERTtokenizer,False,0.741940,0.749476,0.828015,0.818516
BERT,human_enhancers_cohn,DNABERTtokenizer,True,0.726540,0.733819,0.800259,0.789622
DEBERTA,human_enhancers_cohn,DNABERTtokenizer,True,0.730282,0.729191,0.804544,0.799972
BERT,human_enhancers_cohn,DNABERTtokenizer,True,0.728411,0.709558,0.806360,0.800327
DEBERTA,human_enhancers_cohn,Kmer7tokenizer,False,0.729131,0.746361,0.812543,0.802695
BERT,human_enhancers_cohn,Kmer7tokenizer,True,0.727979,0.728370,0.803234,0.793617
DEBERTA,human_enhancers_cohn,Kmer7tokenizer,True,0.720927,0.724456,0.797471,0.788885


In [8]:
dfs[3]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
BERT,human_enhancers_ensembl,DNABERTtokenizer,False,0.866096,0.865867,0.941497,0.942554
BERT,human_enhancers_ensembl,DNABERTtokenizer,True,0.799871,0.811748,0.889409,0.892675
DEBERTA,human_enhancers_ensembl,DNABERTtokenizer,False,0.862577,0.855856,0.940856,0.942380
BERT,human_enhancers_ensembl,DNABERTtokenizer,False,0.866096,0.865867,0.941497,0.942554
BERT,human_enhancers_ensembl,DNABERTtokenizer,True,0.799871,0.811748,0.889409,0.892675
DEBERTA,human_enhancers_ensembl,DNABERTtokenizer,True,0.812851,0.810848,0.893935,0.895679
BERT,human_enhancers_ensembl,DNABERTtokenizer,True,0.805392,0.798906,0.882320,0.884570
DEBERTA,human_enhancers_ensembl,Kmer7tokenizer,False,0.870940,0.872752,0.943192,0.942908
BERT,human_enhancers_ensembl,Kmer7tokenizer,True,0.796384,0.787175,0.872875,0.870909
DEBERTA,human_enhancers_ensembl,Kmer7tokenizer,True,0.795221,0.790319,0.870675,0.870248


In [9]:
dfs[4]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
BERT,human_nontata_promoters,DNABERTtokenizer,False,0.828979,0.815125,0.926020,0.951453
BERT,human_nontata_promoters,DNABERTtokenizer,True,0.823998,0.809398,0.920357,0.947520
DEBERTA,human_nontata_promoters,DNABERTtokenizer,False,0.901041,0.907587,0.955788,0.969006
BERT,human_nontata_promoters,DNABERTtokenizer,False,0.828979,0.815125,0.926020,0.951453
BERT,human_nontata_promoters,DNABERTtokenizer,True,0.823998,0.809398,0.920357,0.947520
DEBERTA,human_nontata_promoters,DNABERTtokenizer,True,0.835510,0.824225,0.924087,0.950049
BERT,human_nontata_promoters,DNABERTtokenizer,True,0.831636,0.819552,0.921892,0.948723
DEBERTA,human_nontata_promoters,Kmer7tokenizer,False,0.907239,0.909989,0.959075,0.972731
BERT,human_nontata_promoters,Kmer7tokenizer,True,0.858424,0.851916,0.944867,0.963659
DEBERTA,human_nontata_promoters,Kmer7tokenizer,True,0.871043,0.867146,0.946988,0.965614


In [10]:
dfs[5]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
BERT,human_ocr_ensembl,DNABERTtokenizer,False,0.768511,0.762344,0.850755,0.839602
BERT,human_ocr_ensembl,DNABERTtokenizer,True,0.714952,0.721322,0.788795,0.775356
DEBERTA,human_ocr_ensembl,DNABERTtokenizer,False,0.770600,0.763829,0.851470,0.841236
BERT,human_ocr_ensembl,DNABERTtokenizer,False,0.768511,0.762344,0.850755,0.839602
BERT,human_ocr_ensembl,DNABERTtokenizer,True,0.714952,0.721322,0.788795,0.775356
DEBERTA,human_ocr_ensembl,DNABERTtokenizer,True,0.718156,0.729480,0.794750,0.782172
BERT,human_ocr_ensembl,DNABERTtokenizer,True,0.713836,0.727302,0.789594,0.774474
DEBERTA,human_ocr_ensembl,Kmer7tokenizer,False,0.771115,0.773165,0.856579,0.847089
BERT,human_ocr_ensembl,Kmer7tokenizer,True,0.726797,0.740552,0.800538,0.784340
DEBERTA,human_ocr_ensembl,Kmer7tokenizer,True,0.723764,0.725624,0.799970,0.786248


In [11]:
dfs[6]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
DEBERTA,drosophila_enhancers_stark,DNABERTtokenizer,True,0.501734,0.044346,0.639391,0.597302
BERT,drosophila_enhancers_stark,DNABERTtokenizer,False,0.703468,0.734884,0.772339,0.733333
BERT,drosophila_enhancers_stark,DNABERTtokenizer,True,0.626012,0.674711,0.672333,0.624246
DEBERTA,drosophila_enhancers_stark,DNABERTtokenizer,False,0.686705,0.738921,0.767109,0.734075
DEBERTA,drosophila_enhancers_stark,Kmer7tokenizer,True,0.599422,0.606026,0.652760,0.616001
DEBERTA,drosophila_enhancers_stark,Kmer7tokenizer,False,0.706358,0.753398,0.777169,0.729518
BERT,drosophila_enhancers_stark,Kmer7tokenizer,True,0.538150,0.681546,0.644930,0.604853
DEBERTA,drosophila_enhancers_stark,Kmer7tokenizer,True,0.501156,0.667181,0.564710,0.551351
BERT,drosophila_enhancers_stark,Kmer8tokenizer,True,0.500000,0.666667,0.508811,0.503915
DEBERTA,drosophila_enhancers_stark,Kmer8tokenizer,True,0.524277,0.674832,0.626897,0.601248


#### The whole table to csv

In [12]:
# main_df.to_csv('all_outputs.csv')

#### Exploration

In [13]:
from genomic_benchmarks.data_check.info import list_datasets

/home/jovyan/my-conda-envs/myEnv/lib/python3.8/site-packages/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [14]:
main_df.keys()

Index(['architecture', 'tokenizer', 'random_weights', 'dataset', 'test_acc',
       'test_f1', 'test_loss', 'test_precision', 'test_recall',
       'test_auroc_macro', 'test_auroc_weighted', 'test_pr_auc',
       'min_valid_loss_epoch', 'min_valid_loss_log', 'file_name'],
      dtype='object')

In [15]:
all_datasets = list_datasets()
all_datasets.remove('dummy_mouse_enhancers_ensembl')
all_datasets

['demo_coding_vs_intergenomic_seqs',
 'human_enhancers_ensembl',
 'human_enhancers_cohn',
 'demo_human_or_worm',
 'drosophila_enhancers_stark',
 'human_ocr_ensembl',
 'human_ensembl_regulatory',
 'human_nontata_promoters']

In [16]:
categories = ['dataset', 'architecture', 'tokenizer', 'random_weights']
exp_options = {}
for category in categories:
    exp_options[category] = []

for category in categories:
    exp_options[category].extend(main_df[category].unique())
exp_options

{'dataset': ['demo_coding_vs_intergenomic_seqs',
  'demo_human_or_worm',
  'human_enhancers_cohn',
  'human_enhancers_ensembl',
  'human_nontata_promoters',
  'human_ocr_ensembl',
  'drosophila_enhancers_stark'],
 'architecture': ['BERT', 'DEBERTA'],
 'tokenizer': ['DNABERTtokenizer',
  'Sentencepiece10tokenizer',
  'Sentencepiece30tokenizer',
  'Kmer7tokenizer',
  'Kmer8tokenizer'],
 'random_weights': [False, True]}

#### Actual and missing combinations of Architecture X Tokenizer X Dataset X Pre-training

In [21]:
all_combinations = []
actual_combinations = []

exptected_len = len(exp_options['dataset']) * len(exp_options['architecture']) * len(exp_options['tokenizer']) * len(exp_options['random_weights'])
print('exptected_len ', exptected_len)

for dataset in exp_options['dataset']:
    for architecture in exp_options['architecture']:
        for tokenizer in exp_options['tokenizer']:
            for random_weights in exp_options['random_weights']:
                all_combinations.append(tuple((dataset, architecture, tokenizer, random_weights)))
        
print('all_combinations ', len(set(all_combinations)))

all_combinations_unique = set(all_combinations)
print('all_combinations_unique ', len(set(all_combinations_unique)), ' should equal exptected_len and all_combinations')

main_df.reset_index()
for index, row in main_df.iterrows():
    row_dic = {}    
    for cat in categories:
        row_dic[cat] = row[cat]
        
    actual_combinations.append(tuple((row['dataset'], row['architecture'], row['tokenizer'], row['random_weights'])))

actual_combinations_unique = set(actual_combinations)

print('actual_combinations ', len(actual_combinations))
print('actual_combinations_unique ', len(actual_combinations_unique))

missing_combinations_unique = all_combinations_unique.difference(actual_combinations_unique)

# remove unwanted combinations -- we are not pre-training out BERT for different tokenizers Kmer7tokenizer
unwanted_combinations = []
for dataset in exp_options['dataset']:
    architecture = 'BERT'
    random_weights = False
    # for tokenizer in exp_options['tokenizer'].remove('DNABERTtokenizer'):
    for tokenizer in ['Kmer7tokenizer', 'Kmer8tokenizer', 'Sentencepiece10tokenizer', 'Sentencepiece30tokenizer']:
        unwanted_combinations.append(tuple((dataset, architecture, tokenizer, random_weights)))

print('unwanted_combinations ', len(unwanted_combinations))
missing_combinations_unique = missing_combinations_unique.difference(unwanted_combinations)

print('missing_combinations_unique ', len(missing_combinations_unique))

actual_combinations_unique = pd.DataFrame(list(actual_combinations_unique), columns=categories)
missing_combinations_unique = pd.DataFrame(list(missing_combinations_unique), columns=categories)
actual_combinations = pd.DataFrame(list(actual_combinations), columns=categories)


exptected_len  140
all_combinations  140
all_combinations_unique  140  should equal exptected_len and all_combinations
actual_combinations  150
actual_combinations_unique  112
unwanted_combinations  28
missing_combinations_unique  0


In [18]:
actual_combinations_unique[actual_combinations_unique['dataset'] == 'drosophila_enhancers_stark']

,dataset,architecture,tokenizer,random_weights
3,drosophila_enhancers_stark,DEBERTA,Sentencepiece10tokenizer,False
5,drosophila_enhancers_stark,DEBERTA,Kmer8tokenizer,False
24,drosophila_enhancers_stark,BERT,DNABERTtokenizer,False
26,drosophila_enhancers_stark,DEBERTA,DNABERTtokenizer,False
27,drosophila_enhancers_stark,DEBERTA,Sentencepiece30tokenizer,True
44,drosophila_enhancers_stark,DEBERTA,Kmer7tokenizer,True
53,drosophila_enhancers_stark,DEBERTA,Sentencepiece30tokenizer,False
67,drosophila_enhancers_stark,DEBERTA,DNABERTtokenizer,True
73,drosophila_enhancers_stark,DEBERTA,Sentencepiece10tokenizer,True
74,drosophila_enhancers_stark,BERT,Kmer7tokenizer,True


In [19]:
actual_combinations_unique[actual_combinations_unique['tokenizer'] == 'Sentencepiece30tokenizer'][actual_combinations_unique['architecture'] == 'BERT']

/tmp/ipykernel_17626/2246666772.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  actual_combinations_unique[actual_combinations_unique['tokenizer'] == 'Sentencepiece30tokenizer'][actual_combinations_unique['architecture'] == 'BERT']


,dataset,architecture,tokenizer,random_weights
20,human_nontata_promoters,BERT,Sentencepiece30tokenizer,True
32,demo_human_or_worm,BERT,Sentencepiece30tokenizer,True
56,human_enhancers_cohn,BERT,Sentencepiece30tokenizer,True
61,human_enhancers_ensembl,BERT,Sentencepiece30tokenizer,True
78,human_ocr_ensembl,BERT,Sentencepiece30tokenizer,True
108,demo_coding_vs_intergenomic_seqs,BERT,Sentencepiece30tokenizer,True
110,drosophila_enhancers_stark,BERT,Sentencepiece30tokenizer,True


In [20]:
# missing_combinations_unique[missing_combinations_unique['dataset'] == 'drosophila_enhancers_stark']
missing_combinations_unique

,dataset,architecture,tokenizer,random_weights
